# Bank Churn Analysis

## Libraries and importing dataset

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style="darkgrid",font_scale=1.5)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score ,f1_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

from imblearn.over_sampling import SMOTE

In [24]:
df = pd.read_csv('Churn_Modelling.csv', index_col=0)
df.head(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Read in the data
1.check the dimensions of dataset

2.fetching and attributes name and know what their meaning

3.generate info of the whole data

4.missing detection

5.duplicated detection

6.descriptive statisical analysis on categorical columns

In [25]:
print("Total number of records/rows present in the dataset is:",df.shape[0])
print("Total number of attributes/columns present in the dataset is:",df.shape[1])

Total number of records/rows present in the dataset is: 10000
Total number of attributes/columns present in the dataset is: 13


In [26]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

Here are some interpretion in some columns

CreditScore: It is a numerical value that assesses the creditworthiness of an individual based on their credit history and financial behavior.

geography: customer geography distribution

tenure: the number of years the customer has been associated with the bank

NumOfProducts:  It can include various offerings such as savings accounts, loans, credit cards, etc.

HasCrCard: It is a binary variable with a value of 1 if the customer possesses a credit card and 0 otherwise.

IsActiveMember: It is a binary variable indicating whether the customer is an active member (1) or not (0) within the bank.

EstimatedSalary: It provides an approximation of the customer's income level, which can be relevant for analyzing churn behavior.

Exited: It indicates whether a customer has churned (1) or not (0) from the bank.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


10000 records in the dataset and non-null records.
10 numerical and 3 categorical

In [28]:
df.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values"})

,Total No. of Missing Values
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0


None missing value in this dataset

In [29]:
df[df.duplicated()]

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,


no duplicate records

In [30]:
df.describe(include="object").T

,count,unique,top,freq
Surname,10000,2932,Smith,32
Geography,10000,3,France,5014
Gender,10000,2,Male,5457


I think Surename has not relationship with the customer churned or not. Unless their name has magic. So I will drop it.

## Data Wrangling
1.dropping insignificant columns
2.Rename some columns for better using 

In [31]:
df.drop(columns=["CustomerId", "Surname"],inplace=True)

In [32]:
df.rename(columns={"Exited":"Churned"},inplace=True)
df['Churned'].replace({0:"No", 1:"Yes"},inplace=True)
df.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churned
RowNumber,,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88,Yes
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,No
3,502,France,Female,42,8,159660.80,3,1,0,113931.57,Yes
4,699,France,Female,39,1,0.00,2,0,0,93826.63,No
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,No
